In [1]:
from turtle import width
import plotly.graph_objects as go
import rospy
import rospkg
import networkx as nx
import xml.etree.ElementTree as ET
from ast import literal_eval
import pandas as pd
import numpy as np
from turtle import width
import plotly.graph_objects as go
import rospy
import rospkg
import networkx as nx
import xml.etree.ElementTree as ET
from ast import literal_eval
import pandas as pd
import numpy as np
from PIL import Image
import os
import urllib.parse



In [2]:
graph_name = 'parliament'
# range = 500
dirname = rospkg.RosPack().get_path('mrpp_sumo')
# no_of_base_stations = np.load(dirname + '/scripts/algorithms/partition_based_patrolling/graphs_partition_results/'+ graph_name + '/required_no_of_base_stations.npy')[0]
graph_results_path = dirname + '/scripts/algorithms/partition_based_patrolling/graphs_partition_results/'

G = nx.read_graphml(dirname + '/graph_ml/' + graph_name + '.graphml')

In [8]:
## Edges of the graph
edge_x = []
edge_y = []

for e in G.edges():
    shape = G[e[0]][e[1]]['shape'].split()
    print(shape)
    for point in shape:
        point = pd.eval(point)
        edge_x.append(point[0])
        edge_y.append(point[1])
    edge_x.append(None)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='black'),
    hoverinfo='none',
    mode='lines')

['-70.85137385455891,1415.1456543854438', '-2.110440294141881,1373.3289336492307']
['-70.85137385455891,1415.1456543854438', '-60.42715263413265,1433.2004518597387', '-51.66199820488691,1452.1002819943242', '-44.612102058832534,1471.7110529867932', '-39.3426081915386,1491.8541594203562', '-35.899720853427425,1512.3846093183383', '-34.679195341072045,1528.2837583632208', '-34.30010246904567,1533.1468737879768', '-34.550224436447024,1553.9634404154494', '-36.6667481737677,1574.678781969007', '-40.62681008898653,1595.1159332073294', '-46.38860890280921,1615.1315426141955', '-53.91950403491501,1634.5488284360617', '-63.14856762893032,1653.236069214996', '-63.14856762893032,1653.236069214996']
['-70.85137385455891,1415.1456543854438', '-165.0872212763643,1472.4651814037934']
['-70.85137385455891,1415.1456543854438', '-70.85137385455891,1415.1456543854438', '-82.72510562359821,1398.2391362097114', '-96.03774748113938,1382.4367513065226', '-110.70356128399726,1367.8620541482233', '-126.597281

In [4]:
## Nodes of the graph
node_x = []
node_y = []
for node in G.nodes():
    node_x.append(G.nodes[node]['x'])
    node_y.append(G.nodes[node]['y'])


node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='Jet',
        reversescale=True,
        color=[],
        size=6,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text




In [6]:
## Plot all data

fig = go.Figure(data=[edge_trace],
             layout=go.Layout(
                title=graph_name +' Communication Graph with '+ str(range) +'m Range IoT devices',
                title_x = 0.5,
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                # annotations=[ dict(
                #     text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                #     showarrow=False,
                #     xref="paper", yref="paper",
                #     x=0.005, y=-0.002 ) ],
                yaxis=dict(scaleanchor="x", scaleratio=1))
                )

# Base stations 
base_stations_df = pd.read_csv(graph_results_path + graph_name + '/' + str(range) + '_range_base_stations.csv',converters={'location': pd.eval,'Radius': pd.eval})
base_station_logo = Image.open(dirname + '/scripts/algorithms/partition_based_patrolling/plot/base.png')

base_stations = []
icons = []
for idx, base_station in base_stations_df.iterrows():
    radius = base_station['Radius']
    location = base_station['location']
    base_stations.append(dict(type="circle",
    xref="x", yref="y",
    fillcolor="rgba(1,1,1,0.1)",
    x0=location[0]-radius, y0=location[1]-radius, x1=location[0]+radius, y1=location[1]+radius,
    line_color="LightSeaGreen",line_width = 0
                    ))

    icons.append(dict(
            source=base_station_logo,
            xref="x",
            yref="y",
            x=location[0]-radius/10,
            y=location[1]+radius/10,
            sizex = radius/5,
            sizey = radius/5
        ))

fig.update_layout(shapes=base_stations, images=icons)


file_name = graph_name + "_" + str(range) + "_range"
file_name = file_name + ".html"
plot_dir = dirname + 'scripts/algorithms/partition_based_patrolling/plot/'+ graph_name + '/communication_graph/'

if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

fig.write_html(plot_dir+file_name)

print("http://vishwajeetiitb.github.io/mrpp_iot//scripts/algorithms/partition_based_patrolling/plot/"+ graph_name + '/communication_graph/' + urllib.parse.quote(file_name))


fig.show()

http://vishwajeetiitb.github.io/mrpp_iot//scripts/algorithms/partition_based_patrolling/plot/parliament/communication_graph/parliament_500_range.html


In [3]:
import random
import math

def select_random_points_on_line_segments(line_segments, n):
    total_length = 0
    cumulative_lengths = []
    for segment in line_segments:
        if len(segment) == 2:
            # Line segment represented by two points
            p1, p2 = segment
            length = math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
        else:
            # Line segment represented by multiple points
            length = 0
            for i in range(len(segment) - 1):
                p1, p2 = segment[i], segment[i+1]
                length += math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
        total_length += length
        cumulative_lengths.append(total_length)

    random_points = []
    for i in range(n):
        # Generate a random number between 0 and the total length of all the line segments
        random_num = random.uniform(0, total_length)

        # Find the index of the line segment that the random point is on
        segment_index = 0
        while segment_index < len(cumulative_lengths) and cumulative_lengths[segment_index] < random_num:
            segment_index += 1

        # Calculate the position of the random point on the line segment
        if len(line_segments[segment_index]) == 2:
            # Line segment represented by two points
            p1, p2 = line_segments[segment_index]
            # print(p1)
            fraction = (random_num - (cumulative_lengths[segment_index] - length)) / length
            x = p1[0] + fraction * (p2[0] - p1[0])
            y = p1[1] + fraction * (p2[1] - p1[1])
        else:
            # Line segment represented by multiple points
            length_so_far = cumulative_lengths[segment_index] - length
            for i in range(len(line_segments[segment_index]) - 1):
                p1, p2 = line_segments[segment_index][i], line_segments[segment_index][i+1]
                segment_length = math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
                if length_so_far + segment_length >= random_num:
                    fraction = (random_num - length_so_far) / segment_length
                    x = p1[0] + fraction * (p2[0] - p1[0])
                    y = p1[1] + fraction * (p2[1] - p1[1])
                    break
                else:
                    length_so_far += segment_length

        random_points.append((x, y))

    return random_points



In [4]:
import plotly.graph_objs as go
import plotly.io as pio

def plot_points(line_segments, points):
    fig = go.Figure()
    for segment in line_segments:
        if len(segment) == 2:
            fig.add_trace(go.Scatter(x=[segment[0][0], segment[1][0]],
                                     y=[segment[0][1], segment[1][1]],
                                     mode='lines',
                                     line=dict(color='black', width=2)))
        else:
            fig.add_trace(go.Scatter(x=[p[0] for p in segment],
                                     y=[p[1] for p in segment],
                                     mode='lines',
                                     line=dict(color='black', width=2)))

    fig.add_trace(go.Scatter(x=[p[0] for p in points],
                             y=[p[1] for p in points],
                             mode='markers',
                             marker=dict(color='red', size=10)))
    fig.update_layout(showlegend=False)
    pio.show(fig)

segments = ['-281.78615814400837,1306.7656111549586', '-280.37454195623286,1352.425457275007', '-279.6467638347531,1377.4953414173797', '-279.385300865164,1386.591446118895', '-279.2774221614236,1390.2632098793983', '-278.8204829469323,1406.2146427384578', '-278.9348181675887,1416.057676997967', '-278.9348181675887,1416.057676997967']

line_segments = []
for segment in segments:
    x, y = segment.split(',')
    line_segments.append((float(x), float(y)))

print(line_segments)


points = select_random_points_on_line_segments(line_segments, 5)
plot_points(line_segments, points)


[(-281.78615814400837, 1306.7656111549586), (-280.37454195623286, 1352.425457275007), (-279.6467638347531, 1377.4953414173797), (-279.385300865164, 1386.591446118895), (-279.2774221614236, 1390.2632098793983), (-278.8204829469323, 1406.2146427384578), (-278.9348181675887, 1416.057676997967), (-278.9348181675887, 1416.057676997967)]


TypeError: 'float' object is not subscriptable

In [1]:
x = range(6)
for n in x:
  print(n)


0
1
2
3
4
5


In [1]:
import numpy as np

def closest_point(x, y, line_segments):
    x, y = np.array([x, y])
    line_segments = np.array(line_segments)

    # Calculate the distances from the point to each line segment
    x1, y1 = line_segments[:, 0, 0], line_segments[:, 0, 1]
    x2, y2 = line_segments[:, 1, 0], line_segments[:, 1, 1]
    dx, dy = x2 - x1, y2 - y1
    t = ((x - x1) * dx + (y - y1) * dy) / (dx * dx + dy * dy)
    t = np.clip(t, 0, 1)
    closest = np.column_stack([x1 + t * dx, y1 + t * dy])
    dists = np.sqrt(np.sum((closest - [x, y]) ** 2, axis=1))

    # Find the index of the closest line segment
    idx = np.argmin(dists)

    return closest[idx]


In [20]:
import random

# Define the closest_point function here (use either the original or optimized version)

# Generate some random line segments and a point
num_segments = 10
line_segments = [((random.uniform(-10, 10), random.uniform(-10, 10)), 
                  (random.uniform(-10, 10), random.uniform(-10, 10)))
                 for i in range(num_segments)]
point = (random.uniform(-10, 10), random.uniform(-10, 10))

# Find the closest point on any of the line segments to the given point
closest = closest_point(point[0], point[1], line_segments)
print(line_segments)
print("Point: ({}, {})".format(*point))
print("Closest point on any line segment: ({}, {})".format(*closest))


[((-7.669726185342462, -2.253090878766006), (2.090020735135985, -4.743958949424347)), ((-4.035165101898189, 0.46610566822029575), (8.108119928525273, 0.1563467112909489)), ((1.929792857886003, 8.783321292081471), (0.6931202594290458, -1.772275883351373)), ((-8.713945833953332, -2.3116623781166146), (9.160863040285438, 8.199134669532345)), ((3.1626042035702575, -6.3461190652996), (-3.975840657411302, 8.774362444825016)), ((4.140553557857293, 3.6880346037576253), (-6.875469413030116, -5.06924161210788)), ((-2.6124503930540133, 3.9022390616755622), (2.6942888052777647, 5.703590175237601)), ((1.486428517571829, -1.3327247666058373), (4.137303097686571, 7.581465013631522)), ((0.6989290198871245, 3.6727835660886967), (-8.643951024891896, 4.604499966474657)), ((2.2385083612322845, -1.9770214011610783), (-3.4835020499469094, -6.148341119173615))]
Point: (7.740496971609126, 2.8076381658648426)
Closest point on any line segment: (7.673149105233168, 0.16744223512820522)


In [10]:
def closest_point(x, y, segments):
    closest = None
    min_dist = float('inf')

    for segment in segments:
        # Parse the x and y coordinates of the segment's endpoints from the string
        x1, y1, x2, y2 = map(float, segment.split(','))

        # Calculate the closest point on the segment to the given point
        dx = x2 - x1
        dy = y2 - y1
        t = ((x - x1) * dx + (y - y1) * dy) / (dx * dx + dy * dy)
        t = max(0, min(1, t))  # Clamp t between 0 and 1
        closest_on_segment = (x1 + t * dx, y1 + t * dy)

        # Calculate the distance between the closest point and the given point
        dist = math.sqrt((closest_on_segment[0] - x) ** 2 + (closest_on_segment[1] - y) ** 2)

        # Update the closest point if this segment is closer
        if dist < min_dist:
            closest = closest_on_segment
            min_dist = dist

    return closest


In [11]:
segments = ['-281.78615814400837,1306.7656111549586', '-280.37454195623286,1352.425457275007', '-279.6467638347531,1377.4953414173797', '-279.385300865164,1386.591446118895', '-279.2774221614236,1390.2632098793983', '-278.8204829469323,1406.2146427384578', '-278.9348181675887,1416.057676997967', '-278.9348181675887,1416.057676997967']

x = 0  # Replace with your x coordinate
y = 0  # Replace with your y coordinate
closest = closest_point(x, y, segments)
print("Closest point on any line segment: ({}, {})".format(*closest))


ValueError: not enough values to unpack (expected 4, got 2)

In [4]:
import numpy as np

segment = ['-281.78615814400837,1306.7656111549586', '-280.37454195623286,1352.425457275007', '-279.6467638347531,1377.4953414173797', '-279.385300865164,1386.591446118895', '-279.2774221614236,1390.2632098793983', '-278.8204829469323,1406.2146427384578', '-278.9348181675887,1416.057676997967', '-278.9348181675887,1416.057676997967']

# Split the segment into individual points
points = np.array([p.split(',') for p in segment], dtype=float)

# Calculate the differences between consecutive points
differences = np.diff(points, axis=0)

# Group the differences into pairs to form line segments
line_segments = zip(points[:-1], differences)


[(array([-281.78615814, 1306.76561115]), array([ 1.41161619, 45.65984612])), (array([-280.37454196, 1352.42545728]), array([ 0.72777812, 25.06988414])), (array([-279.64676383, 1377.49534142]), array([0.26146297, 9.0961047 ])), (array([-279.38530087, 1386.59144612]), array([0.1078787 , 3.67176376])), (array([-279.27742216, 1390.26320988]), array([ 0.45693921, 15.95143286])), (array([-278.82048295, 1406.21464274]), array([-0.11433522,  9.84303426])), (array([-278.93481817, 1416.057677  ]), array([0., 0.]))]
[[-281.78615814 1306.76561115]
 [-280.37454196 1352.42545728]
 [-279.64676383 1377.49534142]
 [-279.38530087 1386.59144612]
 [-279.27742216 1390.26320988]
 [-278.82048295 1406.21464274]
 [-278.93481817 1416.057677  ]
 [-278.93481817 1416.057677  ]]


In [41]:
import numpy as np

def closest_point(x, y, line_segments):
    x, y = np.array([x, y])
    line_segments = np.array(line_segments)

    # Calculate the distances from the point to each line segment
    x1, y1 = line_segments[:, 0, 0], line_segments[:, 0, 1]
    x2, y2 = line_segments[:, 1, 0], line_segments[:, 1, 1]
    dx, dy = x2 - x1, y2 - y1
    # print(dx)
    t = ((x - x1) * dx + (y - y1) * dy) / (dx * dx + dy * dy)
    t = np.clip(t, 0, 1)
    closest = np.column_stack([x1 + t * dx, y1 + t * dy])
    dists = np.sqrt(np.sum((closest - [x, y]) ** 2, axis=1))

    # Find the index of the closest line segment
    idx = np.argmin(dists)

    return closest[idx]

## Edges of the graph
edge_x = []
edge_y = []
segments = []
for e in G.edges():
    edge = G[e[0]][e[1]]['shape'].split()
    points = np.array([p.split(',') for p in edge], dtype=float).tolist()

    lines = [(points[i], points[i+1]) for i in range(len(points)-1)]
    segments.extend(lines)
    
closest_point(1,1,segments)


[ 68.74093356  10.42422122   8.76515443 ... 329.88448751  15.04541623
   0.        ] [-41.81672074  18.05479747  18.89983013 ... -17.40646032  -1.04856398
   0.        ]


/tmp/ipykernel_5840/4166630061.py:12: RuntimeWarning: invalid value encountered in divide
  t = ((x - x1) * dx + (y - y1) * dy) / (dx * dx + dy * dy)


array([nan, nan])

In [43]:
import random

# Define the closest_point function here (use either the original or optimized version)

# Generate some random line segments and a point
num_segments = 2
line_segments = [((random.uniform(-10, 10), random.uniform(-10, 10)), 
                  (random.uniform(-10, 10), random.uniform(-10, 10)))
                 for i in range(num_segments)]
point = (random.uniform(-10, 10), random.uniform(-10, 10))

# Find the closest point on any of the line segments to the given point
closest = closest_point(point[0], point[1], line_segments)
print(line_segments)
print("Point: ({}, {})".format(*point))
print("Closest point on any line segment: ({}, {})".format(*closest))


[ -9.93158643 -10.51325826] [  8.77651403 -10.91650954]
[((2.9463936358761966, -9.48325599651519), (-6.98519279403246, -0.7067419634710514)), ((2.6213826699796545, 0.9290094090568886), (-7.89187559089436, -9.987500129171456))]
Point: (-9.739885841678808, -0.6985367645488303)
Closest point on any line segment: (-6.985192794032459, -0.7067419634710514)
